# Inspecting SageMaker Debugger Logs and Results 

<img align="left" width="130" src="https://raw.githubusercontent.com/PacktPublishing/Amazon-SageMaker-Cookbook/master/Extra/cover-small-padded.png"/>

This notebook contains the code to help readers work through one of the recipes of the book [Machine Learning with Amazon SageMaker Cookbook: 80 proven recipes for data scientists and developers to perform ML experiments and deployments](https://www.amazon.com/Machine-Learning-Amazon-SageMaker-Cookbook/dp/1800567030)

### How to do it...

In [49]:
%store -r artifacts_path
artifacts_path

's3://sagemaker-cookbook-bucket/debugger/sagemaker-xgboost-2021-04-23-12-46-34-431/debug-output'

In [50]:
!pip install smdebug

In [51]:
from smdebug.trials import create_trial

trial = create_trial(artifacts_path)

[2021-04-23 18:09:26.590 ip-172-16-33-253:9310 INFO s3_trial.py:42] Loading trial debug-output at path s3://sagemaker-cookbook-bucket/debugger/sagemaker-xgboost-2021-04-23-12-46-34-431/debug-output
[2021-04-23 18:09:27.926 ip-172-16-33-253:9310 WARNING s3handler.py:183] Encountered the exception ('Connection broken: IncompleteRead(0 bytes read, 268 more expected)', IncompleteRead(0 bytes read, 268 more expected)) while reading s3://sagemaker-cookbook-bucket/debugger/sagemaker-xgboost-2021-04-23-12-46-34-431/debug-output/index/000000001/000000001314_worker_0.json . Will retry now


In [52]:
trial.tensor_names()

['train-error', 'validation-error']

In [53]:
trial.__dict__

{'name': 'debug-output',
 '_tensors': {'train-error': <smdebug.core.tensor.Tensor at 0x7f463b108dd8>,
  'validation-error': <smdebug.core.tensor.Tensor at 0x7f4638da5b00>},
 '_mode_to_global': {},
 '_global_to_mode': {0: (<ModeKeys.GLOBAL: 4>, 0),
  2: (<ModeKeys.GLOBAL: 4>, 2),
  4: (<ModeKeys.GLOBAL: 4>, 4),
  6: (<ModeKeys.GLOBAL: 4>, 6),
  8: (<ModeKeys.GLOBAL: 4>, 8),
  10: (<ModeKeys.GLOBAL: 4>, 10),
  12: (<ModeKeys.GLOBAL: 4>, 12),
  14: (<ModeKeys.GLOBAL: 4>, 14),
  16: (<ModeKeys.GLOBAL: 4>, 16),
  18: (<ModeKeys.GLOBAL: 4>, 18),
  20: (<ModeKeys.GLOBAL: 4>, 20),
  22: (<ModeKeys.GLOBAL: 4>, 22),
  24: (<ModeKeys.GLOBAL: 4>, 24),
  26: (<ModeKeys.GLOBAL: 4>, 26),
  28: (<ModeKeys.GLOBAL: 4>, 28),
  30: (<ModeKeys.GLOBAL: 4>, 30),
  32: (<ModeKeys.GLOBAL: 4>, 32),
  34: (<ModeKeys.GLOBAL: 4>, 34),
  36: (<ModeKeys.GLOBAL: 4>, 36),
  38: (<ModeKeys.GLOBAL: 4>, 38),
  40: (<ModeKeys.GLOBAL: 4>, 40),
  42: (<ModeKeys.GLOBAL: 4>, 42),
  44: (<ModeKeys.GLOBAL: 4>, 44),
  46: (<Mode

In [54]:
trial.__dict__.keys()

dict_keys(['name', '_tensors', '_mode_to_global', '_global_to_mode', 'logger', 'parallel', 'check', 'range_steps', 'collection_manager', 'loaded_all_steps', 'cache', 'path', 'index_reader', 'index_tensors_dict', 'index_mode', 'last_event_token', 'last_index_token', 'worker_set', 'global_step_to_tensors_map', 'mode_to_tensors_map', 'num_workers', 'workers_for_global_step', 'last_complete_step', '_incomplete_wait_for_step_window', 'dynamic_refresh', '_training_end_delay_refresh', 'bucket_name', 'prefix_name'])

In [55]:
trial.bucket_name

'sagemaker-cookbook-bucket'

In [56]:
trial.prefix_name

'debugger/sagemaker-xgboost-2021-04-23-12-46-34-431/debug-output/'

In [57]:
trial.prefix_name

'debugger/sagemaker-xgboost-2021-04-23-12-46-34-431/debug-output/'

In [58]:
target_path = f"s3://{trial.bucket_name}/{trial.prefix_name}"
target_path

In [59]:
s3_contents = !aws s3 ls {target_path} --recursive
s3_contents

['2021-04-23 12:49:35       5504 debugger/sagemaker-xgboost-2021-04-23-12-46-34-431/debug-output/collections/000000000/worker_0_collections.json',
 '2021-04-23 12:49:35        222 debugger/sagemaker-xgboost-2021-04-23-12-46-34-431/debug-output/events/000000000000/000000000000_worker_0.tfevents',
 '2021-04-23 12:49:35        226 debugger/sagemaker-xgboost-2021-04-23-12-46-34-431/debug-output/events/000000000002/000000000002_worker_0.tfevents',
 '2021-04-23 12:49:35        226 debugger/sagemaker-xgboost-2021-04-23-12-46-34-431/debug-output/events/000000000004/000000000004_worker_0.tfevents',
 '2021-04-23 12:49:35        226 debugger/sagemaker-xgboost-2021-04-23-12-46-34-431/debug-output/events/000000000006/000000000006_worker_0.tfevents',
 '2021-04-23 12:49:35        226 debugger/sagemaker-xgboost-2021-04-23-12-46-34-431/debug-output/events/000000000008/000000000008_worker_0.tfevents',
 '2021-04-23 12:49:35        228 debugger/sagemaker-xgboost-2021-04-23-12-46-34-431/debug-output/events

In [60]:
chosen_path = s3_contents[-1].split(" ")[-1]
chosen_path

'debugger/sagemaker-xgboost-2021-04-23-12-46-34-431/debug-output/index/000000006/000000006998_worker_0.json'

In [61]:
chosen_fullpath = f"s3://{trial.bucket_name}/{chosen_path}"
chosen_fullpath

's3://sagemaker-cookbook-bucket/debugger/sagemaker-xgboost-2021-04-23-12-46-34-431/debug-output/index/000000006/000000006998_worker_0.json'

In [62]:
!mkdir -p tmp

In [63]:
!aws s3 cp {chosen_fullpath} tmp/worker_0.json

download: s3://sagemaker-cookbook-bucket/debugger/sagemaker-xgboost-2021-04-23-12-46-34-431/debug-output/index/000000006/000000006998_worker_0.json to tmp/worker_0.json


In [64]:
!cat tmp/worker_0.json

{"meta": {"mode": "GLOBAL", "mode_step": 6998, "event_file_name": "events/000000006998/000000006998_worker_0.tfevents"}, "tensor_payload": [{"tensorname": "train-error", "start_idx": 0, "length": 112}, {"tensorname": "validation-error", "start_idx": 112, "length": 122}]}

In [65]:
trial.tensor("train-error").workers(2)

['worker_0']

In [78]:
for i in [2, 4, 6, 8, 10, 12, 14, 16]:
    traint = trial.tensor("train-error")
    train_error = traint.value(i)[0]
    train_error = "{0:.4f}".format(train_error)
    
    valt = trial.tensor("validation-error")
    validation_error = valt.value(i)[0]
    validation_error = "{0:.4f}".format(validation_error)
    print(f"STEP {i}: [TRAIN ERROR]={train_error} " +
          "[VALIDATION ERROR]={validation_error}")

STEP 2: [TRAIN ERROR]=0.1053 [VALIDATION ERROR]={validation_error}
STEP 4: [TRAIN ERROR]=0.0893 [VALIDATION ERROR]={validation_error}
STEP 6: [TRAIN ERROR]=0.0800 [VALIDATION ERROR]={validation_error}
STEP 8: [TRAIN ERROR]=0.0670 [VALIDATION ERROR]={validation_error}
STEP 10: [TRAIN ERROR]=0.0633 [VALIDATION ERROR]={validation_error}
STEP 12: [TRAIN ERROR]=0.0613 [VALIDATION ERROR]={validation_error}
STEP 14: [TRAIN ERROR]=0.0567 [VALIDATION ERROR]={validation_error}
STEP 16: [TRAIN ERROR]=0.0540 [VALIDATION ERROR]={validation_error}


In [67]:
%store -r rule_job_summary

In [79]:
def lfx(r):
    return r["RuleConfigurationName"] == "LossNotDecreasing"

loss_not_decreasing_summary = list(filter(
    lfx, 
    rule_job_summary))[0]

loss_not_decreasing_summary

{'RuleConfigurationName': 'LossNotDecreasing',
 'RuleEvaluationJobArn': 'arn:aws:sagemaker:us-east-1:581320662326:processing-job/sagemaker-xgboost-2021-04--lossnotdecreasing-ee3df9a4',
 'RuleEvaluationStatus': 'IssuesFound',
 'StatusDetails': 'RuleEvaluationConditionMet: Evaluation of the rule LossNotDecreasing at step 8 resulted in the condition being met\n',
 'LastModifiedTime': datetime.datetime(2021, 4, 23, 12, 57, 6, 274000, tzinfo=tzlocal())}

In [69]:
summary = loss_not_decreasing_summary
rule_evaluation_job_arn = summary['RuleEvaluationJobArn']
rule_evaluation_job_arn

'arn:aws:sagemaker:us-east-1:581320662326:processing-job/sagemaker-xgboost-2021-04--lossnotdecreasing-ee3df9a4'

In [70]:
import sagemaker 
from sagemaker.processing import ProcessingJob

session = sagemaker.Session()
processing_job = ProcessingJob.from_processing_arn(
    sagemaker_session=session, 
    processing_job_arn=rule_evaluation_job_arn)

processing_job

In [71]:
processing_job.__dict__

{'inputs': [<sagemaker.processing.ProcessingInput at 0x7f46388b6710>],
 'outputs': [<sagemaker.processing.ProcessingOutput at 0x7f46388b63c8>],
 'output_kms_key': None,
 'sagemaker_session': <sagemaker.session.Session at 0x7f4638680b70>,
 'job_name': 'sagemaker-xgboost-2021-04--lossnotdecreasing-ee3df9a4'}

In [72]:
processing_job.outputs[0].__dict__

{'source': '/opt/ml/processing/output/rule',
 'destination': 's3://sagemaker-us-east-1-581320662326/sagemaker-xgboost-2021-04-23-12-46-34-431/rule-output/LossNotDecreasing',
 'output_name': 'RuleOutput',
 's3_upload_mode': 'EndOfJob',
 'app_managed': False,
 'feature_store_output': None}

In [73]:
processing_job.job_name

'sagemaker-xgboost-2021-04--lossnotdecreasing-ee3df9a4'

In [74]:
region = "us-east-1"
group = "/aws/sagemaker/ProcessingJobs"
prefix = processing_job.job_name

In [75]:
!pip install awslogs

In [76]:
!awslogs get {group} --log-stream-name-prefix {prefix} -s3h --aws-region {region}

/aws/sagemaker/ProcessingJobs job-ZQRUWYEXTU-ProfilerReport-1619192043-24a126b6/algo-1-1619192262 [2021-04-23 15:38:14.058 ip-10-0-109-175.ec2.internal:1 INFO utils.py:27] RULE_JOB_STOP_SIGNAL_FILENAME: /opt/ml/processing/input/profiler/signals/ProfilerReport-1619192043
/aws/sagemaker/ProcessingJobs job-ZQRUWYEXTU-ProfilerReport-1619192043-24a126b6/algo-1-1619192262 [2021-04-23 15:38:14.524 ip-10-0-109-175.ec2.internal:1 INFO profiler_trial.py:67] Waiting for profiler data.
/aws/sagemaker/ProcessingJobs job-ZQRUWYEXTU-ProfilerReport-1619192043-24a126b6/algo-1-1619192262 [2021-04-23 15:38:24.530 ip-10-0-109-175.ec2.internal:1 INFO profiler_trial.py:37] Output files of ProfilerTrial will be saved to /opt/ml/processing/output/rule
/aws/sagemaker/ProcessingJobs job-ZQRUWYEXTU-ProfilerReport-1619192043-24a126b6/algo-1-1619192262 No environment variable found with name "base_trial". Will use default param value if present
/aws/sagemaker/ProcessingJobs job-ZQRUWYEXTU-ProfilerReport-1619192043

/aws/sagemaker/ProcessingJobs job-HLMHVSHJLD-ProfilerReport-1619192161-0cfb91a9/algo-1-1619192364 ":900},"id":"1250","type":"Paragraph"}],"root_ids":["1250"]},"title":"Bokeh Application","version":"2.2.3"}};\n  var render_items = [{"docid":"f5fb3242-a8df-48d8-abe1-76c67a741b82","root_ids":["1250"],"roots":{"1250":"9d3f08bf-1368-49a2-bbc5-dcfff68efb91"}}];\n  root.Bokeh.embed.embed_items_notebook(docs_json, render_items);\n\n  }\n  if (root.Bokeh !== undefined) {\n    embed_document(root);\n  } else {\n    var attempts = 0;\n    var timer = setInterval(function(root) {\n      if (root.Bokeh !== undefined) {\n        clearInterval(timer);\n        embed_document(root);\n      } else {\n        attempts++;\n        if (attempts > 100) {\n          clearInterval(timer);\n          console.log("Bokeh: ERROR: Unable to run BokehJS code because BokehJS library is missing");\n        }\n      }\n    }, 10, root)\n  }\n})(window);', 'application/vnd.bokehjs_exec.v0+json': ''}}, {'output_type': 

/aws/sagemaker/ProcessingJobs job-UUMPHCNNKS-ProfilerReport-1619192149-2b7264b6/algo-1-1619192357 ary[rule_name][\'Number of datapoints\'] = rule_report[\'Datapoints\']\n    summary[rule_name][\'Rule parameters\'] = rule_report[\'RuleParameters\']\n\ndf = pd.DataFrame.from_dict(summary, orient=\'index\')\ndf = df.sort_values(by=[\'Number of times rule triggered\'], ascending=False)\n\n\ndisplay(Markdown(f"""The following table shows a profiling summary of the Debugger built-in rules. \nThe table is sorted by the rules that triggered the most frequently. During your training job, the {df.index[0]} rule\nwas the most frequently triggered. It processed {df.values[0,3]} datapoints and was triggered {df.values[0,2]} times."""))\n\nwith pd.option_context(\'display.colheader_justify\',\'left\'):    \n    pretty_print(df)'}, {'cell_type': 'code', 'execution_count': 17, 'metadata': {'tags': ['hide-input'], 'papermill': {'exception': False, 'start_time': '2021-04-23T15:40:13.565195', 'end_time':

/aws/sagemaker/ProcessingJobs job-EEIGKCWSSO-ProfilerReport-1619192156-521eb762/algo-1-1619192361 ttempts > 100) {\n          clearInterval(timer);\n          console.log("Bokeh: ERROR: Unable to run BokehJS code because BokehJS library is missing");\n        }\n      }\n    }, 10, root)\n  }\n})(window);', 'application/vnd.bokehjs_exec.v0+json': ''}}], 'source': 'if analyse_phase == "training": \n    display(Markdown("""### CPU bottlenecks\\n\\n"""))\n\n    report = load_report(\'CPUBottleneck\')\n    if report:\n        params = report[\'RuleParameters\'].split(\'\\n\')\n        threshold = int(params[0].split(\':\')[1])\n        cpu_threshold = int(params[1].split(\':\')[1])\n        gpu_threshold = int(params[2].split(\':\')[1])\n        patience = int(params[3].split(\':\')[1])\n        violations = report[\'Violations\']\n        triggered = report[\'RuleTriggered\']\n        datapoints = report[\'Datapoints\']\n        \n        if report[\'Violations\'] > 0:\n            perc =

/aws/sagemaker/ProcessingJobs job-SVKXQJNJXR-ProfilerReport-1619192151-ec8c2af1/algo-1-1619192355 [2021-04-23 15:40:24.925 ip-10-0-244-44.ec2.internal:1 INFO profiler_report.py:106] Putting output notebook in /opt/ml/processing/output/rule/profiler-output/profiler-report.ipynb
/aws/sagemaker/ProcessingJobs job-SVKXQJNJXR-ProfilerReport-1619192151-ec8c2af1/algo-1-1619192355 [2021-04-23 15:40:24.925 ip-10-0-244-44.ec2.internal:1 INFO profiler_report.py:111] Putting html in /opt/ml/processing/output/rule/profiler-output/profiler-report.html
/aws/sagemaker/ProcessingJobs job-SVKXQJNJXR-ProfilerReport-1619192151-ec8c2af1/algo-1-1619192355 [2021-04-23 15:40:25.342 ip-10-0-244-44.ec2.internal:1 INFO rule_invoker.py:34] No more profiler data for rule ProfilerReport at timestamp 1619192460000000
/aws/sagemaker/ProcessingJobs job-SVKXQJNJXR-ProfilerReport-1619192151-ec8c2af1/algo-1-1619192355 [2021-04-23 15:40:25.342 ip-10-0-244-44.ec2.internal:1 INFO rule_invoker.py:41] Ended execution of rule 

...

  if (url in hashes) {\n        element.crossOrigin = "anonymous";\n        element.integrity = "sha384-" + hashes[url];\n      }\n      console.debug("Bokeh: injecting script tag for BokehJS library: ", url);\n      document.head.appendChild(element);\n    }\n  };\n\n  function inject_raw_css(css) {\n    const element = document.createElement("style");\n    element.appendChild(document.createTextNode(css));\n    document.body.appendChild(element);\n  }\n\n  \n  var js_urls = ["https://cdn.bokeh.org/bokeh/release/bokeh-2.2.3.min.js", "https://cdn.bokeh.org/bokeh/release/bokeh-widgets-2.2.3.min.js", "https://cdn.bokeh.org/bokeh/release/bokeh-tables-2.2.3.min.js"];\n  var css_urls = [];\n  \n\n  var inline_js = [\n    function(Bokeh) {\n      Bokeh.set_log_level("info");\n    },\n    function(Bokeh) {\n    \n    \n    }\n  ];\n\n  function run_inline_js() {\n    \n    if (root.Bokeh !== undefined || force === true) {\n      \n    for (var i = 0; i < inline_js.length; i++) {\n      inline

/aws/sagemaker/ProcessingJobs job-ZFZAYEOGFY-ProfilerReport-1619199820-fcec9a82/algo-1-1619200012 [2021-04-23 17:47:50.933 ip-10-0-235-57.ec2.internal:1 INFO profiler_report.py:104] notebook execute return code:{'cells': [{'cell_type': 'markdown', 'metadata': {'tags': [], 'papermill': {'exception': False, 'start_time': '2021-04-23T17:47:47.837487', 'end_time': '2021-04-23T17:47:47.856327', 'duration': 0.01884, 'status': 'completed'}}, 'source': '# SageMaker Debugger Profiling Report\n\nSageMaker Debugger auto generated this report. You can generate similar reports on all supported training jobs. The report provides summary of training job, system resource usage statistics, framework metrics, rules summary, and detailed analysis from each rule. The graphs and tables are interactive. \n\n**Legal disclaimer:** This report and any recommendations are provided for informational purposes only and are not definitive. You are responsible for making your own independent assessment of the inform

/aws/sagemaker/ProcessingJobs job-ZFZAYEOGFY-ProfilerReport-1619199820-fcec9a82/algo-1-1619200012 [2021-04-23 17:47:50.934 ip-10-0-235-57.ec2.internal:1 INFO profiler_report.py:106] Putting output notebook in /opt/ml/processing/output/rule/profiler-output/profiler-report.ipynb
/aws/sagemaker/ProcessingJobs job-ZFZAYEOGFY-ProfilerReport-1619199820-fcec9a82/algo-1-1619200012 [2021-04-23 17:47:50.934 ip-10-0-235-57.ec2.internal:1 INFO profiler_report.py:111] Putting html in /opt/ml/processing/output/rule/profiler-output/profiler-report.html
/aws/sagemaker/ProcessingJobs job-ZFZAYEOGFY-ProfilerReport-1619199820-fcec9a82/algo-1-1619200012 [2021-04-23 17:47:51.348 ip-10-0-235-57.ec2.internal:1 INFO rule_invoker.py:34] No more profiler data for rule ProfilerReport at timestamp 1619200080000000
/aws/sagemaker/ProcessingJobs job-ZFZAYEOGFY-ProfilerReport-1619199820-fcec9a82/algo-1-1619200012 [2021-04-23 17:47:51.348 ip-10-0-235-57.ec2.internal:1 INFO rule_invoker.py:41] Ended execution of rule 

 if (url in hashes) {\n        element.crossOrigin = "anonymous";\n        element.integrity = "sha384-" + hashes[url];\n      }\n      console.debug("Bokeh: injecting script tag for BokehJS library: ", url);\n      document.head.appendChild(element);\n    }\n  };\n\n  function inject_raw_css(css) {\n    const element = document.createElement("style");\n    element.appendChild(document.createTextNode(css));\n    document.body.appendChild(element);\n  }\n\n  \n  var js_urls = ["https://cdn.bokeh.org/bokeh/release/bokeh-2.2.3.min.js", "https://cdn.bokeh.org/bokeh/release/bokeh-widgets-2.2.3.min.js", "https://cdn.bokeh.org/bokeh/release/bokeh-tables-2.2.3.min.js"];\n  var css_urls = [];\n  \n\n  var inline_js = [\n    function(Bokeh) {\n      Bokeh.set_log_level("info");\n    },\n    function(Bokeh) {\n    \n    \n    }\n  ];\n\n  function run_inline_js() {\n    \n    if (root.Bokeh !== undefined || force === true) {\n      \n    for (var i = 0; i < inline_js.length; i++) {\n      inline_

/aws/sagemaker/ProcessingJobs sagemaker-xgboost-2021-04--ProfilerReport-1619200479-3e9cafb8/algo-1-1619200718 [2021-04-23 17:59:25.257 ip-10-2-219-252.ec2.internal:1 INFO profiler_report.py:161] Invoking rule:OverallSystemUsage for timestamp_start:1619200620000000 to timestamp_end:1619200680000000
/aws/sagemaker/ProcessingJobs sagemaker-xgboost-2021-04--ProfilerReport-1619200479-3e9cafb8/algo-1-1619200718 [2021-04-23 17:59:25.262 ip-10-2-219-252.ec2.internal:1 INFO profiler_report.py:161] Invoking rule:OverallFrameworkMetrics for timestamp_start:1619200620000000 to timestamp_end:1619200680000000
/aws/sagemaker/ProcessingJobs sagemaker-xgboost-2021-04--ProfilerReport-1619200479-3e9cafb8/algo-1-1619200718 [2021-04-23 17:59:25.262 ip-10-2-219-252.ec2.internal:1 INFO profiler_report.py:161] Invoking rule:StepOutlier for timestamp_start:1619200620000000 to timestamp_end:1619200680000000
/aws/sagemaker/ProcessingJobs sagemaker-xgboost-2021-04--ProfilerReport-1619200479-3e9cafb8/algo-1-161920